In [1]:
import sys
import config
from config import *

In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [3]:
hot_songs = pd.read_csv('hot_songs.csv')
not_hot_songs = pd.read_csv('not_hot_songs.csv')

In [4]:
hot_songs.head()

,Title,Artists
0,Rockin' Around The Christmas Tree,Brenda Lee
1,All I Want For Christmas Is You,Mariah Carey
2,Jingle Bell Rock,Bobby Helms
3,Last Christmas,Wham!
4,A Holly Jolly Christmas,Burl Ives


In [5]:
not_hot_songs = not_hot_songs.drop(['level_0'], axis=1)
not_hot_songs = not_hot_songs.drop(['index'], axis=1)
not_hot_songs.head()

,Artists,Title
0,Sam Feldt,Post Malone
1,Quality Control,100 Racks (Offset feat. Playboi Carti)
2,Drake,God's Plan
3,Francis and the Lights,May I Have This Dance (Remix)
4,Future,Trapped In The Sun


In [11]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import time
import numpy as np

In [12]:
def search_song(title, artist, limit):
    sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=client_ID,
                                                           client_secret=client_secret))
    try:
        results = sp.search(q=f"{title} {artist}", type='track', limit=limit)
        song_id = results['tracks']['items'][0]['id']
        return song_id
    except IndexError:
        print("Song not found!")
        return ""

def get_audio_features(list_of_song_ids):

    sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=client_ID,
                                                           client_secret=client_secret))

    audio_features_dict = {'id': [], 'danceability': [], 'energy': [],
                           'key': [], 'loudness': [], 'mode': [],
                           'speechiness': [], 'acousticness': [],
                           'instrumentalness': [], 'liveness': [],
                           'valence': [], 'tempo': []}
  
    chunks = np.array_split(list_of_song_ids, len(list_of_song_ids) // 50 + 1)

    for chunk in chunks:
        print("Collecting IDs for chunk...")
        time.sleep(20) 
        features = sp.audio_features(chunk)
        for feature in features:
            for key, value in feature.items():
                if key in audio_features_dict:
                    audio_features_dict[key].append(value)

    audio_features_df = pd.DataFrame(audio_features_dict)
    return audio_features_df

def add_audio_features(df, audio_features_df):
  
    merged_df = pd.merge(df, audio_features_df, on='id', how='inner')
    return merged_df

In [13]:
#from functions import search_song, get_audio_features, add_audio_features

list_of_ids_hot = []
list_of_ids_not_hot = []

for index, row in hot_songs.iterrows():
    try:
        song_id = search_song(row['Title'], row['Artists'], limit=1)
        list_of_ids_hot.append(song_id)
    except:
        print("Error processing song:", row['Title'], row['Artists'])
        list_of_ids_hot.append("")

hot_songs['id'] = list_of_ids_hot

list_of_ids_hot = hot_songs['id'].tolist()
audio_features_hot = get_audio_features(list_of_ids_hot)

hot_songs_feat = add_audio_features(hot_songs, audio_features_hot)

In [14]:
audio_features_hot

,id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,2EjXfH91m7f8HiJN1yQg97,0.589,0.472,8,-8.749,1,0.0502,0.61400,0.000000,0.5050,0.898,67.196
1,7iKRL1F3m4t4dkMponnD0P,0.548,0.871,7,-4.253,1,0.0708,0.00113,0.000000,0.7650,0.640,128.136
2,7vQbuQcyTflfCIOu3Uzzya,0.754,0.424,2,-8.463,1,0.0363,0.64300,0.000000,0.0652,0.806,119.705
3,2FRnf9qhLbvw8fu4IBXx78,0.735,0.478,2,-12.472,1,0.0293,0.18900,0.000002,0.3550,0.947,107.682
4,77khP2fIVhSW23NwxrRluh,0.683,0.375,0,-13.056,1,0.0303,0.57900,0.000000,0.0760,0.888,140.467
...,...,...,...,...,...,...,...,...,...,...,...,...
95,0O3U5iwTbiXCREMkvotJuN,0.616,0.834,9,-3.069,1,0.0664,0.06000,0.000000,0.3380,0.746,151.701
96,2KslE17cAJNHTsI2MI0jb2,0.711,0.809,2,-4.389,0,0.0955,0.04470,0.000000,0.3390,0.816,106.017
97,73zawW1ttszLRgT9By826D,0.498,0.764,4,-5.006,1,0.0309,0.12300,0.000031,0.1190,0.489,147.984
98,2Sy3dDqPwjnTO3PnommJPe,0.844,0.637,10,-6.072,0,0.0325,0.22000,0.000000,0.0459,0.960,125.012


In [15]:
hot_songs_feat

,Title,Artists,id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,Rockin' Around The Christmas Tree,Brenda Lee,2EjXfH91m7f8HiJN1yQg97,0.589,0.472,8,-8.749,1,0.0502,0.61400,0.000000,0.5050,0.898,67.196
1,All I Want For Christmas Is You,Mariah Carey,7iKRL1F3m4t4dkMponnD0P,0.548,0.871,7,-4.253,1,0.0708,0.00113,0.000000,0.7650,0.640,128.136
2,Jingle Bell Rock,Bobby Helms,7vQbuQcyTflfCIOu3Uzzya,0.754,0.424,2,-8.463,1,0.0363,0.64300,0.000000,0.0652,0.806,119.705
3,Last Christmas,Wham!,2FRnf9qhLbvw8fu4IBXx78,0.735,0.478,2,-12.472,1,0.0293,0.18900,0.000002,0.3550,0.947,107.682
4,A Holly Jolly Christmas,Burl Ives,77khP2fIVhSW23NwxrRluh,0.683,0.375,0,-13.056,1,0.0303,0.57900,0.000000,0.0760,0.888,140.467
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,El Amor de Su Vida,Grupo Frontera & Grupo Firme,0O3U5iwTbiXCREMkvotJuN,0.616,0.834,9,-3.069,1,0.0664,0.06000,0.000000,0.3380,0.746,151.701
96,Standing Next To You,Jung Kook,2KslE17cAJNHTsI2MI0jb2,0.711,0.809,2,-4.389,0,0.0955,0.04470,0.000000,0.3390,0.816,106.017
97,Man Made A Bar,Morgan Wallen Featuring Eric Church,73zawW1ttszLRgT9By826D,0.498,0.764,4,-5.006,1,0.0309,0.12300,0.000031,0.1190,0.489,147.984
98,Que Onda,Calle 24 x Chino Pacas x Fuerza Regida,2Sy3dDqPwjnTO3PnommJPe,0.844,0.637,10,-6.072,0,0.0325,0.22000,0.000000,0.0459,0.960,125.012


In [16]:

for index, row in not_hot_songs.iterrows():
    try:
        song_id = search_song(row['Title'], row['Artists'], limit=1)
        list_of_ids_not_hot.append(song_id)
    except:
        print("Error processing song:", row['Title'], row['Artists'])
        list_of_ids_not_hot.append("")

not_hot_songs['id'] = list_of_ids_not_hot

list_of_ids_not_hot = not_hot_songs['id'].tolist()
audio_features_not_hot = get_audio_features(list_of_ids_not_hot)

not_hot_songs_feat = add_audio_features(not_hot_songs, audio_features_not_hot)



In [20]:
not_hot_songs_feat

,Artists,Title,id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,Sam Feldt,Post Malone,36orMWv2PgvnzXsd5CJ0yL,0.590,0.642,7,-3.870,1,0.1220,0.0771,0.000000,0.1050,0.651,107.356
1,Quality Control,100 Racks (Offset feat. Playboi Carti),4T2dHb8XBYHlVj7Dyo1cW9,0.939,0.632,1,-2.978,1,0.1370,0.0606,0.001360,0.1010,0.364,133.979
2,Drake,God's Plan,6DCZcSspjsKoFjzjrWoCdn,0.754,0.449,7,-9.211,1,0.1090,0.0332,0.000083,0.5520,0.357,77.169
3,Francis and the Lights,May I Have This Dance (Remix),0VaIsQoIQY10uxBrOyZ5cQ,0.784,0.435,8,-9.197,1,0.0647,0.4640,0.000005,0.0913,0.865,90.033
4,Future,Trapped In The Sun,6QeUzTuBoasiii8nevDciq,0.701,0.546,6,-5.918,0,0.1330,0.4350,0.000000,0.1880,0.118,151.900
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2515,YoungBoy Never Broke Again,Vette Motors,0HrWshW3SuFJ6nKS9ba9Mh,0.830,0.644,10,-5.431,0,0.4070,0.0298,0.000000,0.1100,0.227,155.103
2516,Topic,Breaking Me,3H7ihDc1dqLriiWXwsc2po,0.789,0.720,8,-5.652,0,0.2180,0.2230,0.000000,0.1290,0.664,122.031
2517,OneRepublic,Wanted,3fT6xqEkC1jm4mHjOyXbKw,0.665,0.539,1,-6.613,1,0.0699,0.1650,0.000000,0.0912,0.666,145.828
2518,Eslabon Armado,Con Tus Besos,6WYB1odDfMPVMyp1QxFInT,0.681,0.664,6,-8.019,0,0.0353,0.3350,0.000000,0.0418,0.861,102.166


In [18]:
not_hot_songs_feat.isna().sum()

Artists             0
Title               0
id                  0
danceability        0
energy              0
key                 0
loudness            0
mode                0
speechiness         0
acousticness        0
instrumentalness    0
liveness            0
valence             0
tempo               0
dtype: int64

In [19]:
not_hot_songs_feat = not_hot_songs_feat.drop_duplicates(subset=['id'])

In [21]:
not_hot_songs_feat.to_csv('not_hot_songs_with_audio.csv', index=False)

In [22]:
hot_songs_feat.to_csv('hot_songs_with_audio.csv', index=False)